In [1]:
import pandas as pd
import numpy as np

In [396]:
df=pd.read_excel('Copy of loan.xlsx')
df.to_csv('Copy of loan.csv')

In [397]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [398]:
df=df.drop('Loan_ID',axis=1)

In [399]:
df['Gender']=df['Gender'].fillna(df['Gender'].mode()[0])
df['Gender']=df['Gender'].apply(lambda x: 1 if x=='Male' else 0)

In [400]:
df['Married']=df['Married'].fillna(df['Married'].mode()[0])
df['Married']=df['Married'].apply(lambda x: 1 if x=='Yes' else 0)

In [401]:
df['Dependents']=df['Dependents'].replace('3+','3')
df['Dependents']=df['Dependents'].fillna(df['Dependents'].mode()[0])

In [402]:
df['Education']=df['Education'].apply(lambda x: 1 if x=='Graduate' else 0)

In [403]:
df['Self_Employed']=df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])
df['Self_Employed']=df['Self_Employed'].apply(lambda x: 1 if x=='Yes' else 0)

In [404]:
df['CoapplicantIncome']=df['CoapplicantIncome'].astype(int)

In [405]:
df['LoanAmount']=df['LoanAmount'].fillna(df['LoanAmount'].mean())

In [406]:
df['Loan_Amount_Term']=df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0])
df['Loan_Amount_Term']=df['Loan_Amount_Term']/12

In [407]:
df['Credit_History']=df['Credit_History'].fillna(df['Credit_History'].mode()[0])

In [408]:
import seaborn as sns
import matplotlib.pyplot as plt

In [409]:
df['Loan_Status']=df['Loan_Status'].apply(lambda x: 1 if x=='Y' else 0)

In [410]:
df=pd.get_dummies(df,columns=['Property_Area'],drop_first=True)

In [411]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Semiurban,Property_Area_Urban
0,1,0,0,1,0,5849,0,146.412162,30.0,1.0,1,False,True
1,1,1,1,1,0,4583,1508,128.000000,30.0,1.0,0,False,False
2,1,1,0,1,1,3000,0,66.000000,30.0,1.0,1,False,True
3,1,1,0,0,0,2583,2358,120.000000,30.0,1.0,1,False,True
4,1,0,0,1,0,6000,0,141.000000,30.0,1.0,1,False,True


In [412]:
df=df.astype(int)

In [413]:
dff=df.corr()['Loan_Status'].abs().sort_values(ascending=False)
dff

Loan_Status                1.000000
Credit_History             0.540556
Property_Area_Semiurban    0.136540
Married                    0.091478
Education                  0.085884
CoapplicantIncome          0.059188
Property_Area_Urban        0.043621
LoanAmount                 0.036345
Loan_Amount_Term           0.022549
Gender                     0.017987
Dependents                 0.010118
ApplicantIncome            0.004710
Self_Employed              0.003700
Name: Loan_Status, dtype: float64

In [414]:
from sklearn.feature_selection import SelectKBest, chi2

In [415]:
from sklearn.model_selection import train_test_split

In [ ]:
data=df.copy()
df['LoanAmountlog']=np.log(df['LoanAmount'])
q1=np.quantile(df['LoanAmount'],0.25)
q3=np.quantile(df['LoanAmount'],0.75)
IQR=q3-q1
l_b=q1-1.5*IQR
u_b=q3+1.5*IQR
data=df[(df['LoanAmount']>=l_b) & (df['LoanAmount']<=u_b)]

In [570]:
x=data.drop('Loan_Status',axis=1)
y=data['Loan_Status']

In [571]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [516]:
ft=SelectKBest(chi2,k=7)
ttrain=ft.fit_transform(x_train,y_train)
ttest=ft.transform(x_test)

In [517]:
sc=ft.scores_

In [518]:
top7=pd.DataFrame({'name':x_train.columns,'score':sc})
top7.sort_values(by='score',ascending=False)

,name,score
6,CoapplicantIncome,7542.660756
7,LoanAmount,38.320143
9,Credit_History,21.500281
10,Property_Area_Semiurban,5.486751
5,ApplicantIncome,3.543442
8,Loan_Amount_Term,1.252414
1,Married,0.802414
3,Education,0.768208
2,Dependents,0.395798
11,Property_Area_Urban,0.289762


In [421]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [572]:
model=DecisionTreeClassifier(max_depth=2,criterion='entropy')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [423]:
from sklearn.metrics import accuracy_score

In [573]:
ms=accuracy_score(y_test,y_pred)
ms

0.8869565217391304

In [574]:
X=x.drop(columns=['Gender','Self_Employed','Loan_Amount_Term','Education','Dependents','Married'])

In [575]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [581]:
model=DecisionTreeClassifier(max_depth=2,criterion='log_loss')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
ms=accuracy_score(y_test,y_pred)
ms

0.8869565217391304

In [594]:
ml=RandomForestClassifier(n_estimators=150,max_depth=4,max_features=0.5,max_samples=0.7,random_state=2)
ml.fit(x_train,y_train)
y_pred=ml.predict(x_test)
ms=accuracy_score(y_test,y_pred)
ms

0.8869565217391304

In [599]:
from sklearn.naive_bayes import GaussianNB

In [601]:
from xgboost import XGBClassifier

In [608]:
xg=XGBClassifier(n_estimators=10,max_depth=2)
xg.fit(x_train,y_train)
y_pred=xg.predict(x_test)
ms=accuracy_score(y_test,y_pred)
ms

0.8869565217391304

In [ ]:
#ml is final model.

In [609]:
import joblib

In [610]:
joblib.dump(ml,'Loan_ml.pkl')

['Loan_ml.pkl']